# Programmieraufgabe: CG-Verfahren und PCG-Verfahren

Wie in Beispiel 3.39 im Skript betrachten wir ein quadratisches Optimierungsproblem, das aus der zweidimensionalen Wärmeleitungsgleichung resultiert. Dabei hat die Matrix $A$ die Form
$$
 A = \begin{bmatrix}
    B  & -I     &        &    \\
    -I & \ddots & \ddots &    \\
       & \ddots & \ddots & -I \\
       &        &     -I & B
 \end{bmatrix} \in \mathbb{R}^{n \times n}
 $$
 mit 
 $$
 B = \begin{bmatrix}
    4  & -1     &        &    \\
    -1 & \ddots & \ddots &    \\
       & \ddots & \ddots & -1 \\
       &        &     -1 & 4
    \end{bmatrix} \in \mathbb{R}^{\sqrt{n}\times \sqrt{n}}.
$$
Ziel ist es, dieses System mithilfe des **CG-Verfahrens** (konjugierte Gradienten) und des **PCG-Verfahrens** (präkonditionierte Gradienten) zu lösen.

Tragen Sie zunächst in der folgenden Zelle **beide** Ihre Namen ein: 

In [ ]:
# Numerische Optimierungsverfahren der Wirtschaftsmathematik
# Wintersemester 2025/2026
# Übungsblatt 6 - Programmieraufgabe 6
#
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]
# 
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]

In [ ]:
import math
import time
import numpy as np
from numpy.linalg import norm
from numpy.linalg import inv
import matplotlib.pyplot as plt
import sys

## Teil 1: Implementierung der Hilfsfunktionen

Für großes $n$ ist es sehr ineffizient, die Matrix $A$ und den Präkonditionierer $S$ zu speichern. Daher wollen wir die Anwendung dieser Matrizen direkt als Funktionen implementieren. Dieser Ansatz wird auch als "Matrix-Free" bezeichnet.

* `Av = mul_A(v)`: Diese Funktion soll die Multiplikation der Matrix $A$ mit einem beliebigen Vektor $v$ effizient realisieren, **ohne** dass $A$ als $n \times n$-Matrix gespeichert werden muss.
* `w  = mul_pre(r)`: Analog soll diese Funktion die Multiplikation eines Vektors $r$ mit $S^{-T} S^{-1}$ realisieren.

Für die Präkonditionierung soll, wie in der Vorlesung besprochen, $S = \frac{1}{2} D + L^T$ verwendet werden, wobei die Zerlegung $A = L + D + L^T$ zugrunde liegt ($L$ ist der strikte obere Dreiecksteil und $D$ die Diagonale von $A$). 

$S^{-T}S^{-1}$ kann dann leicht mittel Vorwärts- und Rückwärtssubstitution angewendet werden.

In [ ]:
def mul_A(v):
    """
    A*v = mul_A(v)
    Legen Sie hier nicht die Matrix A an!
    """
    sqrt_n = int(np.sqrt(v.shape[0]))
    Av = 4.0 * v
    Av[:-1] -= v[1:]
    Av[1:] -= v[:-1]
    Av[:-sqrt_n] -= v[sqrt_n:]
    Av[sqrt_n:] -= v[:-sqrt_n]
    return Av

In [ ]:
def mul_pre(r):
    """
    Anwendung des Vorkonditionierers S^(-1)S^(-T) auf r
    (Auch hier dürfen weder A noch S^(-1) als Matrix angelegt werden)
    """
    n = r.shape[0]
    sqrt_n = int(np.sqrt(n))
    # z = S^(-T)*r
    z = np.zeros(n)
    z[0] = r[0] / 2.0
    for i in range(1, n):
        z[i] = (r[i] + z[i-1]) / 2.0
        if i >= int(sqrt_n):
            z[i] += z[i - sqrt_n] / 2.0
    # w = S^(-1)*z
    w = np.zeros(n)
    w[-1] = z[-1] / 2.0
    for i in range(2, n+1):
        w[-i] = (z[-i] + w[-(i-1)]) / 2.0
        if i >= int(sqrt_n) + 1:
            w[-i] += w[-(i - sqrt_n)] / 2.0
    return w

Sie können die folgenden beiden Zeilen zum Testen Ihrer Funktionen verwenden:

In [ ]:
m = 4
n = 16
A = 4.0 * np.eye(n) - (np.diag(np.ones(n-1), k=-1) + np.diag(np.ones(n-1), k=1))
A -= (np.diag(np.ones(n - m), k = m) + np.diag(np.ones(n - m), k = -m))
x = np.ones(n)
print(A @ x)
print(mul_A(x))
assert (A @ x == mul_A(x)).all()

In [ ]:
m = 4
n = 16
S =  2.0 * np.eye(n) - np.diag(np.ones(n-1), k=1)
S -= np.diag(np.ones(n - m), k = m)
b = np.ones(n)
print(inv(S) @ inv(S.T) @ b)
print(mul_pre(b))
assert (inv(S) @ inv(S.T) @ b == mul_pre(b)).all()

### Teil 2: Implementierung der CG-Verfahren

Als Nächstes werden die Hauptfunktionen für die iterativen Lösungsverfahren benötigt:

* `iterates, residuals = cg(b, mul_A, tol, kmax)`
* `iterates, residuals = pcg(b, mul_A, mul_pre, tol, kmax)`

Diesen Funktionen werden `mul_A` und `mul_pre` als Parameter übergeben, um die Matrixoperationen durchzuführen.

Parameter:
* `tol` ist die Fehlertoleranz. Die Verfahren sollen abgebrochen werden, wenn **sowohl** die Norm des Residuums **als auch** die Norm der Differenz aufeinanderfolgender Iterierter $\leq$ `tol` ist.
* `kmax` ist die maximale Anzahl an Iterationen. Die Verfahren müssen spätestens terminieren, wenn diese Grenze erreicht ist.
* Als Startvektor wählen Sie die rechte Seite $b$.


Die Rückgabewerte der Funktionen `cg` und `pcg` sollen dabei sein:
* `iterates`: Die vom jeweiligen Verfahren berechneten Iterierten, wobei `iterates[-1,:]` die berechnete Lösung ist.
* `residuals` (Vektor): Der Vektor mit den Normen des Residuums:
    $$\text{res}(i) = \| b - A x^{(i)} \|_{2}$$


In [ ]:
def cg(b, mul_A, tol, max_iter):
    """
    Konjugierte-Gradienten-Verfahren (CG)
        iterates, residuals = cg(b, mul_A, tol, max_iter)

    Parameter:
      b        : Rechte Seite und Initialschätzung (Startvektor x0);
      mul_A    : Funktion, die A*v berechnet;
      tol      : Geforderte absolute Genauigkeit;
      max_iter : Maximale Zahl von Iterationen.

    Resultate:
      iterates  : Folge der Iterierten x^(k)
      residuals : Folge der Residuennormen ||b - A*x^(i)||_2;

    Das Verfahren bricht ab, wenn entweder die maximale Zahl von
    Iterationsschritten 'kmax' erreicht ist oder die 2-Normen von
    Residuum (A*x - b) und Schrittweite (x^(i+1) - x^(i)) beide kleiner-gleich
    der Toleranz 'tol' sind.
    """
    n = b.shape[0]

    residuals = np.zeros(max_iter + 1)  # Residuum  || A*x^(i) - b ||_2
    
    # Initialisierung
    iter_count = 0                           # Zählt die berechneten Iterationsschritte
    iterates = np.zeros((max_iter + 1, n))
    iterates[0,:] = np.copy(b)               # x_0 = b
    residual = b - mul_A(iterates[0,:])           # r_0 = b - A*x_0
    search_direction = np.copy(residual)     # p_0 = r_0
    gamma_old = np.inner(residual, residual) # gamma_0 = <r_0, r_0>

    residuals[iter_count] = math.sqrt(gamma_old) 

    while (iter_count < max_iter) and (
        (residuals[iter_count] > tol) or (step_norm > tol)
    ):
        iter_count += 1 
        # 1. Berechne A*p_k
        v = mul_A(search_direction)
        
        # 2. Berechne alpha_k (Schrittweite)
        omega = np.inner(search_direction, v) # <p_k, A*p_k>
        
        #if abs(omega) < 1e-14:
        #    print(
        #        f"CG-Warnung: k={iter_count}/{max_iter} , gamma_old={format(gamma_old,'.2e')} , <p_k,v_k>={format(omega,'.2e')} < 1e-14 !!"
        #    )
        #    break

        alpha = gamma_old / omega # alpha_k = gamma_k / omega_k

        # 3. Update x_k+1
        step_vector = alpha * search_direction
        step_norm = norm(step_vector)
        iterates[iter_count,:] = iterates[iter_count - 1,:] + step_vector

        # 4. Update r_k+1
        residual -= alpha * v

        # 5. Berechne gamma_k+1
        gamma_new = np.inner(residual, residual) # gamma_k+1 = <r_k+1, r_k+1>

        # 6. Update p_k+1
        beta = gamma_new / gamma_old # beta_k = gamma_k+1 / gamma_k
        
        search_direction *= beta
        search_direction += residual
        
        # 7. Speichern und Update
        
        # Residuen-Norm für den Abbruch: ||r_k+1||_2
        residuals[iter_count] = norm(residual)
        
        gamma_old = gamma_new

    # Die resultierenden Vektoren sollen genau die richtige Länge haben;
    # alle überschüssigen Elemente werden abgeschnitten:
    residuals = residuals[: iter_count + 1]
    iterates = iterates[:iter_count + 1,:]

    return iterates, residuals

In [ ]:
def pcg(b, mul_A, mul_pre, tol, max_iter):
    """
    Präkonditioniertes Konjugierte-Gradienten-Verfahren (PCG)
        iterates, residuals = pcg(b, mul_A, mul_pre, tol, max_iter)

    Parameter:
      b        : Rechte Seite und Initialschätzung (Startvektor x0);
      mul_A    : Funktion, die A*v berechnet;
      mul_pre  : Funktion, die M^(-1)*v berechnet (M^(-1) = S^(-T)*S^(-1));
      tol      : Geforderte absolute Genauigkeit;
      max_iter : Maximale Zahl von Iterationen.

    Resultate:
      iterates  : Folge der Iterierten x^(k)
      residuals : Folge der Residuennormen ||b - A*x^(i)||_2;

    Das Verfahren bricht ab, wenn entweder die maximale Zahl von
    Iterationsschritten 'kmax' erreicht ist oder die 2-Normen von
    Residuum (A*x - b) und Schrittweite (x^(i+1) - x^(i)) beide kleiner-gleich
    der Toleranz 'tol' sind.
    """
    n = b.shape[0]

    # Initialisierung der Ergebnis-Vektoren (mit Platz für den Startwert, Index 0)
    residuals = np.zeros(max_iter + 1)  # Residuum  || A*x^(i) - b ||_2

    # Initialisierung
    iter_count = 0                     # Zählt die berechneten Iterationsschritte
    iterates = np.zeros((max_iter + 1, n))
    iterates[0,:] = np.copy(b)           # x_0 = b
    residual = b - mul_A(iterates[0,:])       # r_0 = b - A*x_0
    
    # Präkonditionierung: w_0 = M^(-1) * r_0
    preconditioned_residual = mul_pre(residual) # w_0 = M^(-1) * r_0
    
    # Suchrichtung: p_0 = w_0
    search_direction = np.copy(preconditioned_residual) 
    
    # Skalarprodukt: gamma_0 = <w_0, r_0>
    gamma_old = np.dot(preconditioned_residual, residual) 

    # Speichern der Werte für Iteration k=0
    residuals[iter_count] = norm(residual) 

    # Iteration
    while (iter_count < max_iter) and (
        (residuals[iter_count] > tol) or (step_norm > tol)
    ):
        iter_count += 1
        # 1. Berechne A*p_k
        vec_v = mul_A(search_direction)
        
        # 2. Berechne alpha_k (Schrittweite)
        # omega = <p_k, A*p_k>
        omega = np.dot(search_direction, vec_v) 
        
        alpha = gamma_old / omega # alpha_k = gamma_k / omega_k

        # 3. Update x_k+1
        step_vector = alpha * search_direction
        step_norm = norm(step_vector)
        iterates[iter_count,:] = iterates[iter_count - 1,:] + step_vector

        # 4. Update r_k+1
        residual -= alpha * vec_v
        
        # 5. Präkonditionierung: w_k+1 = M^(-1) * r_k+1
        preconditioned_residual = mul_pre(residual)
        
        # 6. Berechne gamma_k+1mul
        gamma_new = np.dot(preconditioned_residual, residual) # gamma_k+1 = <w_k+1, r_k+1>

        # 7. Update p_k+1
        beta = gamma_new / gamma_old # beta_k = gamma_k+1 / gamma_k
        
        search_direction *= beta
        search_direction += preconditioned_residual
        

        # Residuen-Norm für den Abbruch: ||r_k+1||_2
        residuals[iter_count] = norm(residual)
        
        gamma_old = gamma_new

    # Die resultierenden Vektoren sollen genau die richtige Länge haben;
    # alle überschuessigen Elemente werden daher abgeschnitten:
    residuals = residuals[:iter_count + 1]
    iterates = iterates[:iter_count + 1,:]

    return iterates, residuals

Sie können die folgenden beiden Zeilen zum Testen Ihrer Funktionen verwenden. Dabei sollte `iterates.shape` die Form `(anzahl_iterationen, 25)` und `residuals.shape` die Form `(anzahl_iterationen,)` haben.

In [ ]:
iterates, residuals = cg(np.ones(5*5), mul_A, 1e-5, 20)
print(iterates.shape)
print(residuals.shape)

In [ ]:
iterates, residuals = pcg(np.ones(5*5), mul_A, mul_pre, 1e-5, 20)
print(iterates.shape)
print(residuals.shape)

### Teil 3: Vergleich der Verfahren

Nun wollen wir die implementierten Verfahren vergleichen. Dazu wenden wir die Funktionen `cg` und `pcg` auf eine Matrix $A \in \mathbb{R}^{n \times n}$ an, wobei $n = m^2$ für ein $m \in \mathbb{N}$.

In [ ]:
from util.plotting_06 import compute_and_plot_06

maxit = 5000
tol = 1e-15
m = 50

b, cg_solution, pcg_solution = compute_and_plot_06(cg, pcg, m, mul_A, mul_pre, tol, maxit)

Zum Schluss können wir auch noch unsere Lösung der Wärmeleitgleichung plotten:

In [ ]:
plt.figure()
plt.subplot(121)
plt.imshow(b.reshape(m,m))
plt.title("$b$")
plt.subplot(122)
plt.imshow(pcg_solution.reshape(m, m))
plt.title("$A^{-1}b$")
plt.show()